<a href="https://colab.research.google.com/github/zatyhan/spam-text-classification/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-02-20 10:24:04--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   131MB/s    in 0.2s    

2024-02-20 10:24:04 (131 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
users_above_200= df_ratings['user'].value_counts()
books_above_100= df_ratings['isbn'].value_counts()

df= df_ratings[df_ratings['user'].isin(users_above_200[users_above_200>=200].index) & df_ratings['isbn'].isin(books_above_100[books_above_100>=100].index)]
df.head()

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0


In [ ]:
new_df= df.pivot_table(columns=['user'], index= ['isbn'],values=['rating']).fillna(0)
new_df


rating                                                          \
user       254    2276   2766   2977   3363   4017   4385   6242   6251     
isbn                                                                        
002542730X    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060008032    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060096195    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
006016848X    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060173289    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
1573227331    0.0    0.0    0.0    0.0    0.0    0.0    0.0    6.0    0.0   
1573229326    0.0    0.0    0.0    0.0    0.0    0.0    0.0    6.0    0.0   
1573229571    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1592400876    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1878424319    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                   ...                                                   \
user       6323    ... 274004 274061 274301 274308 274808 275970 277427   
isbn               ...                                                    
002542730X    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0   10.0   
0060008032    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060096195    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
006016848X    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
0060173289    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...           ...  ...    ...    ...    ...    ...    ...    ...    ...   
1573227331    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1573229326    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1573229571    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1592400876    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1878424319    0.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                                 
user       277478 277639 278418  
isbn                             
002542730X    0.0    0.0    0.0  
0060008032    0.0    0.0    0.0  
0060096195    0.0    0.0    0.0  
006016848X    0.0    0.0    0.0  
0060173289    0.0    0.0    0.0  
...           ...    ...    ...  
1573227331    0.0    0.0    0.0  
1573229326    0.0    0.0    0.0  
1573229571    0.0    0.0    0.0  
1592400876    0.0    0.0    0.0  
1878424319    0.0    0.0    0.0  

[731 rows x 888 columns]

In [ ]:
new_df.index = new_df.join(df_books.set_index('isbn'))['title']
new_df

<ipython-input-124-918305bb1fa5>:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  new_df.index = new_df.join(df_books.set_index('isbn'))['title']


rating                \
user                                               254    2276   2766     
title                                                                     
Politically Correct Bedtime Stories: Modern Tal...    0.0    0.0    0.0   
Angels                                                0.0    0.0    0.0   
The Boy Next Door                                     0.0    0.0    0.0   
Men Are from Mars, Women Are from Venus: A Prac...    0.0    0.0    0.0   
Divine Secrets of the Ya-Ya Sisterhood : A Novel      0.0    0.0    0.0   
...                                                   ...    ...    ...   
About a Boy                                           0.0    0.0    0.0   
How to Be Good                                        0.0    0.0    0.0   
About a Boy (Movie Tie-In)                            0.0    0.0    0.0   
Eats, Shoots &amp; Leaves: The Zero Tolerance A...    0.0    0.0    0.0   
The Four Agreements: A Practical Guide to Perso...    0.0    0.0    0.0   

                                                                         \
user                                               2977   3363   4017     
title                                                                     
Politically Correct Bedtime Stories: Modern Tal...    0.0    0.0    0.0   
Angels                                                0.0    0.0    0.0   
The Boy Next Door                                     0.0    0.0    0.0   
Men Are from Mars, Women Are from Venus: A Prac...    0.0    0.0    0.0   
Divine Secrets of the Ya-Ya Sisterhood : A Novel      0.0    0.0    0.0   
...                                                   ...    ...    ...   
About a Boy                                           0.0    0.0    0.0   
How to Be Good                                        0.0    0.0    0.0   
About a Boy (Movie Tie-In)                            0.0    0.0    0.0   
Eats, Shoots &amp; Leaves: The Zero Tolerance A...    0.0    0.0    0.0   
The Four Agreements: A Practical Guide to Perso...    0.0    0.0    0.0   

                                                                         \
user                                               4385   6242   6251     
title                                                                     
Politically Correct Bedtime Stories: Modern Tal...    0.0    0.0    0.0   
Angels                                                0.0    0.0    0.0   
The Boy Next Door                                     0.0    0.0    0.0   
Men Are from Mars, Women Are from Venus: A Prac...    0.0    0.0    0.0   
Divine Secrets of the Ya-Ya Sisterhood : A Novel      0.0    0.0    0.0   
...                                                   ...    ...    ...   
About a Boy                                           0.0    6.0    0.0   
How to Be Good                                        0.0    6.0    0.0   
About a Boy (Movie Tie-In)                            0.0    0.0    0.0   
Eats, Shoots &amp; Leaves: The Zero Tolerance A...    0.0    0.0    0.0   
The Four Agreements: A Practical Guide to Perso...    0.0    0.0    0.0   

                                                           ...                \
user                                               6323    ... 274004 274061   
title                                                      ...                 
Politically Correct Bedtime Stories: Modern Tal...    0.0  ...    0.0    0.0   
Angels                                                0.0  ...    0.0    0.0   
The Boy Next Door                                     0.0  ...    0.0    0.0   
Men Are from Mars, Women Are from Venus: A Prac...    0.0  ...    0.0    0.0   
Divine Secrets of the Ya-Ya Sisterhood : A Novel      0.0  ...    0.0    0.0   
...                                                   ...  ...    ...    ...   
About a Boy                                           0.0  ...    0.0    0.0   
How to Be Good                                        0.0  ...    0.0    0.0   
About a Boy (Movie

In [ ]:
book= "Where the Heart Is (Oprah's Book Club (Paperback))"
neigh= NearestNeighbors(n_neighbors=6, metric='cosine').fit(new_df)
distance, indices= neigh.kneighbors([new_df.loc[book].values], n_neighbors=6)
recommended_books=[]
for i in range(1,6):
  recommended_books.append([new_df.index[indices[0][-i]], distance[0][-i]])
[book, recommended_books]

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.8016211],
  ['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075],
  ['The Lovely Bones: A Novel', 0.7234864]]]

In [ ]:
neigh= NearestNeighbors(n_neighbors=6).fit(new_df)

frozenset()

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  neigh= NearestNeighbors(n_neighbors=6, metric='cosine').fit(new_df)
  distance, indices= neigh.kneighbors([new_df.loc[book].values], n_neighbors=6)
  recommended_books=[]
  for i in range(1,6):
    recommended_books.append([new_df.index[indices[0][-i]], distance[0][-i]])
  return [book, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
